In [ ]:
import os
import cv2
import mediapipe as mp
from tqdm import tqdm  # прогресс-бар

# Используем только FaceMesh
mp_face_mesh = mp.solutions.face_mesh

# Цвет маски (серый)
MASK_COLOR = (127, 127, 127)

# Индексы ключевых landmark по FaceMesh
LANDMARKS = {
    "mouth": [61, 291, 17, 0],      # левый угол губ, правый угол губ, нижняя губа, верхняя губа
    "eyes": [33, 133, 362, 263],    # левый глаз (33-133), правый глаз (362-263)
    "nose": [1, 2, 98, 327],        # центр носа, кончик и крылья
    "lower_face": [17, 152]         # нижняя губа и подбородок
}

def mask_face_part(image, face_landmarks, part="mouth", color=MASK_COLOR):
    h, w, _ = image.shape

    if part == "mouth":
        x1 = int(0)
        x2 = int(w)
        y1 = int(face_landmarks.landmark[LANDMARKS["mouth"][3]].y * h * 0.9)
        y2 = int(h)
        cv2.rectangle(image, (x1, y1), (x2, y2), color, -1)

    elif part == "eyes":
        # Левый глаз
        x1 = int(face_landmarks.landmark[33].x * w)
        x2 = int(face_landmarks.landmark[133].x * w)
        y1 = int(face_landmarks.landmark[159].y * h)
        y2 = int(face_landmarks.landmark[145].y * h)
        cv2.rectangle(image, (x1, y1 - 10), (x2, y2 + 10), color, -1)

        # Правый глаз
        x1 = int(face_landmarks.landmark[362].x * w)
        x2 = int(face_landmarks.landmark[263].x * w)
        y1 = int(face_landmarks.landmark[386].y * h)
        y2 = int(face_landmarks.landmark[374].y * h)
        cv2.rectangle(image, (x1, y1 - 10), (x2, y2 + 10), color, -1)

    elif part == "nose":
        x1 = int(face_landmarks.landmark[98].x * w)
        x2 = int(face_landmarks.landmark[327].x * w)
        y1 = int(face_landmarks.landmark[1].y * h)
        y2 = int(face_landmarks.landmark[2].y * h)
        cv2.rectangle(image, (x1, y1), (x2, y2), color, -1)

    elif part == "lower_face":
        lip_y = int(face_landmarks.landmark[17].y * h)
        cv2.rectangle(image, (0, lip_y), (w, h), color, -1)

    return image


def process_images(input_folder, output_folder, part="mouth"):
    os.makedirs(output_folder, exist_ok=True)

    # Собираем список файлов заранее
    file_list = [f for f in os.listdir(input_folder) if f.lower().endswith((".jpg", ".png", ".jpeg"))]

    with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1) as face_mesh:
        for filename in tqdm(file_list, desc=f"Processing {part}"):
            path = os.path.join(input_folder, filename)
            image = cv2.imread(path)
            if image is None:
                print(f"[Warning] Cannot read image: {filename}")
                continue

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(image_rgb)

            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    image = mask_face_part(image, face_landmarks, part=part)

            cv2.imwrite(os.path.join(output_folder, filename), image)


if __name__ == "__main__":
    for train_test in ["train", "test"]:
        for i in range(1, 8):

            raf_ds_path = f"datasets/raf/dataset/{train_test}/{i}"
            proc_ds_path = f"datasets/raf/PROC_masked_mouth/{train_test}/{i}"

            process_images(raf_ds_path, os.path.join(proc_ds_path, "masked_mouth"), part="mouth")
            
            # process_images(raf_ds_path, os.path.join(proc_ds_path, "masked_eyes"), part="eyes")
            # process_images(raf_ds_path, os.path.join(proc_ds_path, "masked_nose"), part="nose")
            # process_images(raf_ds_path, os.path.join(proc_ds_path, "masked_lower_face"), part="lower_face")

            print(f"Обработаны изображения: {raf_ds_path} -> {proc_ds_path}")